# Database

In [1]:
import os
import glob
import json
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [2]:
from castor import get_header

In [4]:
from ipywidgets import IntProgress
from IPython.display import display

In [5]:
files = glob.glob("/mnt/arch/POLLUX/flat/*.txt")

## Data from header

In [7]:
prog = IntProgress(max=len(files))
display(prog)
data = pd.DataFrame()
for f in files:
    head = get_header(f)
    head["basename"] = os.path.basename(f).rstrip(".txt")
    data = data.append(head, ignore_index=True)
    prog.value += 1

IntProgress(value=0, max=16921)

In [8]:
data.replace("irrelevant", np.nan, inplace=True)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16921 entries, 0 to 16920
Columns: 339 entries, Abu_01 to line_list_9_type
dtypes: float64(237), object(102)
memory usage: 43.8+ MB


## Export database

In [10]:
engine = create_engine("sqlite:///pollux.db")

In [11]:
data.to_sql("flat", engine, if_exists="replace")

## Keywords list

In [12]:
comments = {}
for f in files:
    head, comm = get_header(f, cmt=True)
    for key in comm.keys():
        if key not in comments:
            comments[key] = comm[key]
comments["basename"] = "base name of the file"

In [13]:
with open("keywords.json", "w") as keys:
    json.dump(comments, keys)